# Gradio GeoLlama App
This notebook shows how to launch the Gradio based GeoLlama App. We will include the full pipeline, from translation to toponym extraction to geoparsing.

In [6]:
import sys
sys.path.append('..')
import gradio as gr
from geo_llama.translator import Translator
from geo_llama.model import RAGModel, TopoModel
from geo_llama.geo_llama import GeoLlama
from geo_llama.plotting import plot_map

In [ ]:
translator = Translator(model_size='1.2B')
topo_model = TopoModel(model_name='JoeShingleton/GeoLlama_7b_toponym', 
                    prompt_path='data/prompt_templates/prompt_template.txt',
                    instruct_path='data/prompt_templates/topo_instruction.txt',
                    input_path=None,
                    config_path='data/config_files/model_config.json')

rag_model = RAGModel(model_name='JoeShingleton/GeoLlama_7b_RAG', 
                    prompt_path='data/prompt_templates/prompt_template.txt',
                    instruct_path='data/prompt_templates/rag_instruction.txt',
                    input_path='data/prompt_templates/rag_input.txt',
                    config_path='data/config_files/model_config.json')

geo_llama = GeoLlama(topo_model, rag_model)

In [ ]:
def translate(text):
    out = translator.translate(text, out_lang='en')
    return out

def geoparse(text):
    translated_text = translate(text, translator)
    locations = geo_llama.geoparse(translated_text['translation'])
    locations_str = ', '.join([x['name'] for x in locations])
    mapped = plot_map(locations)
    return translated_text, locations_str, mapped

In [ ]:
input = gr.Textbox(label='Text')
output1 = gr.Textbox(label='Translation')
output2 = gr.Textbox(label='Toponyms')
output3 = gr.Plot(label='Mapped Locations')
demo = gr.Interface(fn=geoparse, 
                    inputs=input, 
                    outputs=[output1, output2, output3])
demo.launch()